In [117]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [118]:
df = pd.read_excel("data\Full Data.xlsx")

In [119]:
df.columns

Index(['หมายเลขคำสั่งซื้อ', 'สถานะการสั่งซื้อ', 'สถานะการคืนเงินหรือคืนสินค้า',
       'ชื่อผู้ใช้ (ผู้ซื้อ)', 'วันที่ทำการสั่งซื้อ', 'เวลาการชำระสินค้า',
       'ช่องทางการชำระเงิน', 'ช่องทางการชำระเงิน (รายละเอียด)',
       'แผนการผ่อนชำระ', 'ค่าธรรมเนียม (%)', 'ตัวเลือกการจัดส่ง',
       'วิธีการจัดส่ง', '*หมายเลขติดตามพัสดุ',
       'วันที่คาดว่าจะทำการจัดส่งสินค้า', 'เวลาส่งสินค้า',
       'เลขอ้างอิง Parent SKU', 'ชื่อสินค้า',
       'เลขอ้างอิง SKU (SKU Reference No.)', 'ชื่อตัวเลือก', 'ราคาตั้งต้น',
       'ราคาขาย', 'จำนวน', 'Returned quantity', 'ราคาขายสุทธิ',
       'ส่วนลดจาก Shopee', 'โค้ดส่วนลดชำระโดยผู้ขาย', 'โค้ด Coins Cashback',
       'โค้ดส่วนลดชำระโดย Shopee', 'โค้ดส่วนลด',
       'เข้าร่วมแคมเปญ bundle deal หรือไม่',
       'ส่วนลด bundle deal ชำระโดยผู้ขาย', 'ส่วนลด bundle deal ชำระโดย Shopee',
       'ส่วนลดจากการใช้เหรียญ', 'ส่วนลดทั้งหมดจากบัตรเครดิต', 'ค่าคอมมิชชั่น',
       'Transaction Fee', 'ต้นทุนขายหักคูปองและcoin',
       'ค่าจัดส่งที่ชำระโดยผู้ซื้อ', 'ค

**Feature Engineering**

In [120]:
df.drop(columns = ["ค่าธรรมเนียม (%)","วิธีการจัดส่ง" , "เลขอ้างอิง Parent SKU" , "สถานะการสั่งซื้อ","สถานะการคืนเงินหรือคืนสินค้า",
                   "ชื่อผู้รับ" , "หมายเลขโทรศัพท์" , "ที่อยู่ในการจัดส่ง" , "หมายเลขคำสั่งซื้อ", "เวลาที่ทำการสั่งซื้อสำเร็จ"] 
                   , inplace = True)

In [121]:
#Time Preprocess
df["วันที่ทำการสั่งซื้อ"] = pd.to_datetime(df["วันที่ทำการสั่งซื้อ"])
df["เวลาการชำระสินค้า"] = pd.to_datetime(df["เวลาการชำระสินค้า"])

In [122]:
# Example for Hour of the Day
df['Hour'] = df['วันที่ทำการสั่งซื้อ'].dt.hour
df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)

# Example for Day of the Week
df['Day of Week'] = df["วันที่ทำการสั่งซื้อ"].dt.dayofweek
df['DayOfWeek_sin'] = np.sin(2 * np.pi * df['Day of Week'] / 7)
df['DayOfWeek_cos'] = np.cos(2 * np.pi * df['Day of Week'] / 7)

In [123]:
# Example for Time of Day Binning
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['Time of Day'] = df['Hour'].apply(categorize_time_of_day)

# Convert the Time of Day feature to one-hot encoding
df = pd.get_dummies(df, columns=['Time of Day'], drop_first=True)


In [124]:
df = pd.get_dummies(df, columns=["ช่องทางการชำระเงิน"], drop_first=True)

In [125]:
df[["โค้ดส่วนลด"]].fillna(False)

,โค้ดส่วนลด
0,BUILBBB52
1,BUILBBB52
2,BUILBBB52
3,BUILBBB52
4,BUILBBB52
...,...
16600,FSV-946894897442817
16601,FSV-946894897442817
16602,FSV-946894897442821
16603,FSV-946894897442821


In [126]:
def discount_code(x):
    if type(x) == str:
        return True
    else:
        return False

df["โค้ดส่วนลด"] = df["โค้ดส่วนลด"].apply(discount_code)

In [127]:
customer_frequency = df["ชื่อผู้ใช้ (ผู้ซื้อ)"].value_counts()

df["Purchase frequency"] = df["ชื่อผู้ใช้ (ผู้ซื้อ)"].map(customer_frequency)

df[["Purchase frequency"]]

,Purchase frequency
0,1.0
1,3.0
2,3.0
3,3.0
4,3.0
...,...
16600,4.0
16601,4.0
16602,3.0
16603,3.0


In [128]:
df.columns

Index(['ชื่อผู้ใช้ (ผู้ซื้อ)', 'วันที่ทำการสั่งซื้อ', 'เวลาการชำระสินค้า',
       'ช่องทางการชำระเงิน (รายละเอียด)', 'แผนการผ่อนชำระ',
       'ตัวเลือกการจัดส่ง', '*หมายเลขติดตามพัสดุ',
       'วันที่คาดว่าจะทำการจัดส่งสินค้า', 'เวลาส่งสินค้า', 'ชื่อสินค้า',
       'เลขอ้างอิง SKU (SKU Reference No.)', 'ชื่อตัวเลือก', 'ราคาตั้งต้น',
       'ราคาขาย', 'จำนวน', 'Returned quantity', 'ราคาขายสุทธิ',
       'ส่วนลดจาก Shopee', 'โค้ดส่วนลดชำระโดยผู้ขาย', 'โค้ด Coins Cashback',
       'โค้ดส่วนลดชำระโดย Shopee', 'โค้ดส่วนลด',
       'เข้าร่วมแคมเปญ bundle deal หรือไม่',
       'ส่วนลด bundle deal ชำระโดยผู้ขาย', 'ส่วนลด bundle deal ชำระโดย Shopee',
       'ส่วนลดจากการใช้เหรียญ', 'ส่วนลดทั้งหมดจากบัตรเครดิต', 'ค่าคอมมิชชั่น',
       'Transaction Fee', 'ต้นทุนขายหักคูปองและcoin',
       'ค่าจัดส่งที่ชำระโดยผู้ซื้อ', 'ค่าจัดส่งที่ Shopee ออกให้โดยประมาณ',
       'ค่าจัดส่งสินค้าคืน', 'ค่าบริการ', 'จำนวนเงินทั้งหมด',
       'ค่าจัดส่งโดยประมาณ', 'หมายเหตุจากผู้ซื้อ', 'ประเทศ', 'จังหวัด',
       

In [129]:
df.rename(columns = {"ราคาขาย": "Sale Price",
                     "ส่วนลดจาก Shopee": "Shopee Discount",
                     "โค้ด Coins Cashback": "Code Coins Cashback",
                     "ส่วนลด bundle deal ชำระโดย Shopee": "Shopee-Paid Bundle Deal Discount",
                     "โค้ดส่วนลดชำระโดยผู้ขาย": "Discount Code by Seller"
                     }, inplace = True)

In [130]:
customer_groups = [
    {
        "group_name": "Price-Sensitive Customers",
        "relevant_features": [
            "Sale Price", #(Sale Price)
            "Shopee Discount", #(Shopee Discount)
            "Discount Code by Seller", #(Discount Code by Seller)
            "Code Coins Cashback",
        ],
        "description": "Customers who are likely to respond to discounts, cashback offers, and other price-related promotions."
    },
    {
        "group_name": "Product-Focused Customers",
        "relevant_features": [
            #"ชื่อสินค้า", #(Product Name)
            "เลขอ้างอิง SKU (SKU Reference No.)",
            "Shopee-Paid Bundle Deal Discount", 
        ],
        "description": "Customers who are influenced by specific product promotions, bundles, or installment payment options."
    },
    {
        "group_name": "Loyalty-Driven Customers",
        "relevant_features": [
            "Purchase frequency",
            #"Hour_sin",
            "Sale Price",
            "Shopee-Paid Bundle Deal Discount", 
        ],
        "description": "Customers who respond to loyalty rewards and have a history of repeat purchases."
    },
    {
        "group_name": "Convenience-Oriented Customers",
        "relevant_features": [
            "Sale Price", #(Sale Price)
            "Shopee Discount", #(Shopee Discount)
            "Hour_sin"
        ],
        "description": "Customers who prioritize convenience and are influenced by promotions. Theses customers are also influenced by time of the day."
    }
]


In [131]:
category = ["Price-Sensitive Customers" , "Product-Focused Customers" , "Loyalty-Driven Customers" , "Convenience-Oriented Customers" , "High-Value Customers"]

In [132]:
categories = []
for group in customer_groups:
    categories.append(df[group["relevant_features"]])

In [133]:
# categories[0].head()

**Making a readable file**

In [134]:
# with pd.ExcelWriter('Clean.xlsx') as writer:
#     df.to_excel(writer , index = False)

In [135]:
with pd.ExcelWriter('Category.xlsx') as writer:
    for i in range(len(categories)):
        categories[i].to_excel(writer , index = False , sheet_name =  customer_groups[i]["group_name"])